In [ ]:
%pip install langchain_openai

In [1]:
import os
from langchain_openai import OpenAI,ChatOpenAI # type: ignore


os.environ["OPENAI_API_BASE"] ='http://10.35.151.101:8001/v1'
os.environ["OPENAI_API_KEY"] = "sk-1234"
os.environ["OPENAI_MODEL_NAME"] = 'ibm-llama3-1-70b'

model = ChatOpenAI(
    model="ibm-llama3-1-70b",
    temperature=0.3,
    max_tokens=3000,
)

In [3]:
%pip install mysql-connector-python
%pip install python-dotenv
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

True

In [3]:
connection = mysql.connector.connect(
    host="db4free.net",
    user="jerrin23",
    password="9890614803",
    port="3306",
    database="college_database",
    ssl_disabled=True 
)

In [4]:
if connection.is_connected():
    print("Connection Successful!!")
    cursor = connection.cursor()
    try:
        cursor.execute("Select * from students")
        rows = cursor.fetchall()

        for row in rows:
            print(row)
    except mysql.connector.Error as e:
        print("Error reading data from MySQL table : ", e)
    finally:
        cursor.close()
        connection.close()
        print("Connection closed!!")
else:
    print("Connection failed!!")

Connection Successful!!


(1, 'Jasmine', 'Chennai')
(2, 'Rhea', 'Mumbai')
(3, 'Rohit', 'Chennai')
(4, 'Deepak', 'Delhi')
Connection closed!!


In [5]:
def connect_to_database():
    return mysql.connector.connect(
        host="db4free.net",
        user=username,
        password=password,
        database=database_name,
        port='3306',
        ssl_disabled=True
    )

In [6]:
def execute_query(connection, query, database, fetch_data=True):
    try:
        if connection.is_connected():
            cursor = connection.cursor()
            print(query)
            cursor.execute(f"USE {database};")
            cursor.execute(query)
            rows = cursor.fetchall()
            if rows is not None:
                connection.commit()
            return rows
    except mysql.connector.Error as e:
        print("Error executing SQL Query : {e}")
        connection.rollback()

In [9]:
def get_table_data(connection, table_name):
    try:
        cursor = connection.cursor()
        cursor.execute(f"SELECT * FROM {table_name}")
        table_data = cursor.fetchall()
        column_names = [i[0] for i in cursor.description]
        return table_data, column_names
    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return [], []

In [10]:
def get_tables_in_database(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("SHOW TABLES")
        tables = cursor.fetchall()
        table_names = [table[0] for table in tables]
        return table_names
    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return []

In [13]:
database_name = input("Enter the name of the database")
table_name = input("Enter the name of the table")
username = input("Enter your username")
password = input("Enter your password")
column_names = []


try:
    connection = connect_to_database()
    if connection.is_connected():
        tables = get_tables_in_database(connection)
        for table in tables:
            print(f"Table: {table}")
            table_data, column_names = get_table_data(connection, table)
            df = pd.DataFrame(table_data, columns=column_names)
            print(df)
        cursor = connection.cursor()
except mysql.connector.Error as e:
    print("Error reading data from MySQL table:", e)

if database_name and username and password:
    text_input = input("Enter your Query : ")
    if text_input:
        template = f"""
            Create a SQL query snippet using the below text for the table with database name and table name as {database_name}.{table_name} and columns {column_names}:
            ```
                {text_input}
            ```
                Just write the correct SQL Query ending with ';'
            """
        formatted_template = template.format(text_input=text_input)
        response = model(template)
        sql_query = response.content
        print(sql_query)
        rows = execute_query(connection,sql_query,database_name)
        print(rows)
        if rows:
            pd.DataFrame(rows)
        else:
            print("Data updated succesfully!!")
            if connection.is_connected():
                cursor.execute(f'''SELECT * FROM {table_name}''')
                rows = cursor.fetchall()
                pd.DataFrame(rows)

Table: students
   id     name  address
0   1  Jasmine  Chennai
1   2     Rhea   Mumbai
2   3    Rohit  Chennai
INSERT INTO college_database.students (id, name, address) VALUES (4, 'Deepak', 'Delhi');
INSERT INTO college_database.students (id, name, address) VALUES (4, 'Deepak', 'Delhi');
[]
Data updated succesfully!!
